In [1]:
import geojson

import gaia

# Initialize Girder connection
apikey = 'bimEnIEt2RU8U1GqJKkqeJ5G5eH36kTSHGLjaUeA'
datastore = gaia.connect(apikey=apikey)


default_folder: {'size': 0, 'parentCollection': 'folder', 'baseParentId': '5bb7d66f3b4e9a0007762157', 'created': '2018-10-05T22:09:59.398000+00:00', 'updated': '2018-10-05T22:09:59.398000+00:00', 'parentId': '5bb7e0e23b4e9a013ce51766', '_id': '5bb7e1373b4e9a013ce51767', '_accessLevel': 2, 'baseParentType': 'user', 'description': 'Created by Gaia', 'creatorId': '5bb7d66f3b4e9a0007762157', '_modelType': 'folder', 'public': False, 'name': 'default'}


In [2]:
import json

# Create gaia dataset for girder-hosted file
path = 'Public/DEM_bare_earth.tif'  # (~4GB)

# API option 1: create internal url
datastore_url = datastore.lookup_url('Public/DEM_bare_earth.tif')
print('datastore url:', datastore_url)
dataset = gaia.create(datastore_url)

# API option 2: pass in tuple as the data source
# data_source = (datastore, 'Public/DEM_bare_earth.tif')
# dataset = gaia.create(data_source)

meta = dataset.get_metadata()
print()
#print(json.dumps(meta, sort_keys=True, indent=2))
print('Input dataset width: {}, height: {}'.format(meta['width'], meta['height']))
gaia.show(dataset, representation='outline')

datastore url: girder://item/5bb7d7533b4e9a0007762163


/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)



Input dataset width: 28448, height: 39816


<jupyterlab_geojs.Scene object>

In [3]:
# Generate small crop geometry
bounds = meta.get('bounds',{}).get('coordinates')[0]
assert bounds, 'Dataset bounds missing'
# print()
# print(bounds)

# Compute center coordinates
x = (bounds[0][0] + bounds[2][0]) / 2.0
y = (bounds[0][1] + bounds[2][1]) / 2.0

# Use smll percentage of height & width
dx = 0.005 * (bounds[2][0] - bounds[0][0])
dy = 0.005 * (bounds[2][1] - bounds[0][1])
rect = [
    [x,y], [x, y-dy], [x-dx, y-dy], [x-dx, y], [x,y]
]
# print()
# print(rect)

# Must pass rectangle in as a LIST, in order to get geom formatted the way resgeodata uses
crop_geom = geojson.Polygon([rect])
print()
print(crop_geom)



{"coordinates": [[[-106.9669195224971, 38.94564248418757], [-106.9669195224971, 38.944732188346215], [-106.96771495471891, 38.944732188346215], [-106.96771495471891, 38.94564248418757], [-106.9669195224971, 38.94564248418757]]], "type": "Polygon"}


In [5]:
# Crop the dataset
from gaia import preprocess
cropped_dataset = preprocess.crop2(dataset, crop_geom, name='crop100m.tif')

results_folder_id 5bb7e1373b4e9a013ce51767
<generator object GirderClient.listResource at 0x7f1990e8dbf8>
Deleting existing (item crop100m.tif)
 1 Checking job 5bb7e3b03b4e9a013ce51770 status... 2
 2 Checking job 5bb7e3b03b4e9a013ce51770 status... 2
 3 Checking job 5bb7e3b03b4e9a013ce51770 status... 2
 4 Checking job 5bb7e3b03b4e9a013ce51770 status... 2
 5 Checking job 5bb7e3b03b4e9a013ce51770 status... 2
 6 Checking job 5bb7e3b03b4e9a013ce51770 status... 2
 7 Checking job 5bb7e3b03b4e9a013ce51770 status... 2
 8 Checking job 5bb7e3b03b4e9a013ce51770 status... 2
 9 Checking job 5bb7e3b03b4e9a013ce51770 status... 3


In [6]:
print()
cropped_meta = cropped_dataset.get_metadata()
print('Cropped dataset width {}, height {}'.format(
    cropped_meta['width'], cropped_meta['height']))
#print(cropped_meta)


Cropped dataset width 143, height 207


In [7]:
# Todo Update gaia.show() for girder-resident datasets
scene = gaia.show(cropped_dataset, representation="outline")
scene

<jupyterlab_geojs.Scene object>